In [1]:
import pandas as pd
import numpy as np

In [2]:
train=pd.read_csv("Train_cars.csv")
test=pd.read_csv("Test_cars.csv")
ss=pd.read_csv("Sample_Submission.csv")

In [3]:
comb=pd.concat([train,test],axis=0)
comb.isnull().sum()

Name                     0
Location                 0
Year                     0
Kilometers_Driven        0
Fuel_Type                0
Transmission             0
Owner_Type               0
Mileage                341
Engine                 106
Power                  317
Seats                  338
New_Price            61934
Price                52500
dtype: int64

In [4]:
comb.head()
    
comb["Brand"]=[i[0] for i in comb["Name"].str.split()]

In [5]:
comb["M1"]=[i[1] for i in comb["Name"].str.split()]
comb["M2"]=[i[2] for i in comb["Name"].str.split()]
comb["M3"]=[i[-1] for i in comb["Name"].str.split()]

In [6]:
comb.drop("Name",axis=1,inplace=True)

In [7]:
comb["Location"].value_counts()
def loc(i):
    if i in ["Delhi", "Mumbai", "Bangalore", "Chennai", "Hyderabad", "Kolkata", "Ahmedabad","Pune"]:
        return 1
    else:
        return 0
    
comb["Location"]=comb["Location"].apply(loc)

In [8]:
comb["old"]=2022-comb.Year

In [9]:
comb.drop("Year",axis=1,inplace=True)

In [10]:
comb=pd.concat([comb,pd.get_dummies(comb.Fuel_Type,drop_first=True)],axis=1)

In [11]:
comb.drop("Fuel_Type",axis=1,inplace=True)

In [12]:
comb.Transmission.value_counts()
def trans(i):
    if i=="Manual":
        return 0
    else:
        return 1
    
comb["Transmission"]=comb.Transmission.apply(trans)

In [13]:
comb.head()
comb.Owner_Type.value_counts()
def own(i):
    if i=="Fourth & Above":
        return 0
    elif i=="Third":
        return 1
    elif i=="Second":
        return 2
    else:
        return 3
    
comb.Owner_Type=comb.Owner_Type.apply(own)

In [14]:
comb.Mileage=pd.to_numeric(comb.Mileage.replace(regex=["kmpl","km/kg"],value=""))

In [15]:
comb.head()
comb.Engine=pd.to_numeric(comb.Engine.replace(regex=["CC"],value=""))

In [16]:
comb.head()
comb.Power=pd.to_numeric(comb.Power.replace(regex=["bhp","null"],value=["",np.NaN]))

In [17]:
comb.head()
#comb.New_Price=
comb.New_Price=pd.to_numeric(comb.New_Price.replace(regex=["Lakh","Cr"],value=""))

In [18]:
pd.get_dummies(comb.Brand,drop_first=True)
from sklearn.decomposition import PCA
pca=PCA(n_components=15)
brand_pca=pd.DataFrame(pca.fit_transform(pd.get_dummies(comb["Brand"],drop_first=True)))
sum(pca.explained_variance_ratio_)

0.9514151476265318

In [19]:
comb.M3.value_counts()
pca=PCA(n_components=5)
model_pca=pd.DataFrame(pca.fit_transform(pd.get_dummies(comb[["M1","M2","M3"]],drop_first=True)),columns=["MP1","MP2","MP3","MP4","MP5"])
sum(pca.explained_variance_ratio_)

0.11564393498326209

In [20]:
#pd.concat([comb,brand_pca.reset_index()],axis=1)
df=pd.concat([pd.concat([comb.reset_index(),brand_pca.reset_index()],axis=1),model_pca],axis=1).drop(["Brand","M1","M2","M3"],axis=1)

In [21]:
df=df.drop("index",axis=1)

In [22]:
df.iloc[:,:10].corr()
df=df.sort_values(["Transmission","Seats","Engine"]).drop("New_Price",axis=1)

In [ ]:
#df=df.ffill().bfill().reset_index().drop("index",axis=1)
from sklearn.impute import KNNImputer
knn=KNNImputer(n_neighbors=5)
df=knn.fit_transform(df)

G:\anaconda\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
G:\anaconda\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [ ]:
df.head()
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaled=pd.DataFrame(scaler.fit_transform(df[["Kilometers_Driven","Mileage","Engine","Power"]]),columns=["Kilometers_Driven_sc","Mileage_sc","Engine_sc","Power_sc"])

In [ ]:
df1=pd.concat([df,scaled],axis=1).drop(["Kilometers_Driven","Mileage","Engine","Power"],axis=1)

In [ ]:
df1.head()
train.shape
df_train=df1.iloc[:22500,:]
df_test=df1.iloc[22500:,:]
xtrain=df_train.drop("Price",axis=1)
ytrain=df_train["Price"]
xtest=df_test.drop("Price",axis=1)



In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
x1,x2,y1,y2=train_test_split(xtrain,ytrain,test_size=0.25)
lin=LinearRegression()
model=lin.fit(x1,y1)
ypred=model.predict(x2)
mean_squared_error(y2,ypred)

In [ ]:
jbkj

In [ ]:
lin=LinearRegression()
from lightgbm import LGBMRegressor
lgb=LGBMRegressor()
model=lgb.fit(xtrain,ytrain)
preds=model.predict(xtest)


In [ ]:
sub=pd.DataFrame({"Name":test.Name,"Price":preds})
sub.to_csv("c2.csv",index=False)